# Phase 3: Correlation Analysis & Geomagnetic Event Detection

This notebook analyzes Kp index data to:
1. Calculate comprehensive statistics
2. Detect geomagnetic storm events
3. Generate a storm catalog
4. Visualize temporal patterns and storm activity

**Data Period:** November 12-15, 2025

---

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Libraries loaded successfully!")

In [ ]:
# Load the Kp index data
df = pd.read_csv('../data/Space_Weather_Indices_Subset.csv')

# Convert time column to datetime
df['Time (UTC)'] = pd.to_datetime(df['Time (UTC)'], format='%d-%m-%Y %H:%M')

print("✓ Dataset loaded successfully!")
print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Date range: {df['Time (UTC)'].min()} to {df['Time (UTC)'].max()}")
print(f"Time span: {(df['Time (UTC)'].max() - df['Time (UTC)'].min()).days + 1} days")

In [ ]:
# Display first few rows
df.head()

## 2. Kp Index Statistics Analysis

The Kp index is a global geomagnetic activity index ranging from 0 (quiet) to 9 (extreme storm).
We analyze the median Kp values to understand the overall geomagnetic activity during this period.

In [ ]:
# Calculate comprehensive statistics for median Kp
kp_median = df['median']

stats = {
    'Mean': kp_median.mean(),
    'Median': kp_median.median(),
    'Std Dev': kp_median.std(),
    'Min': kp_median.min(),
    'Max': kp_median.max(),
    '25th Percentile': kp_median.quantile(0.25),
    '75th Percentile': kp_median.quantile(0.75),
    '90th Percentile': kp_median.quantile(0.90),
    '95th Percentile': kp_median.quantile(0.95)
}

print("="*60)
print("KP INDEX STATISTICS (Median Values)")
print("="*60)
for key, value in stats.items():
    print(f"{key:20s}: {value:6.3f}")
print("="*60)

In [ ]:
# Additional statistics for all quantiles
print("\nSTATISTICS FOR ALL QUANTILES")
print("="*60)
quantile_cols = ['minimum', '0.25-quantile', 'median', '0.75-quantile', 'maximum']

for col in quantile_cols:
    print(f"\n{col}:")
    print(f"  Mean: {df[col].mean():.3f}")
    print(f"  Range: {df[col].min():.3f} to {df[col].max():.3f}")

## 3. Event Detection Based on Kp Thresholds

Geomagnetic storms are classified by Kp index:
- **Minor (G1)**: Kp = 5 - 5.66
- **Moderate (G2)**: Kp = 6 - 6.66
- **Strong (G3)**: Kp = 7 - 7.66
- **Severe (G4)**: Kp = 8 - 8.66
- **Extreme (G5)**: Kp = 9

In [ ]:
# Define storm classification function
def classify_kp_level(kp_value):
    """Classify Kp index into storm categories."""
    if kp_value < 5:
        return 'Quiet to Active'
    elif kp_value < 6:
        return 'Minor Storm (G1)'
    elif kp_value < 7:
        return 'Moderate Storm (G2)'
    elif kp_value < 8:
        return 'Strong Storm (G3)'
    elif kp_value < 9:
        return 'Severe Storm (G4)'
    else:
        return 'Extreme Storm (G5)'

# Apply classification
df['storm_category'] = df['median'].apply(classify_kp_level)

# Count events by category
event_counts = df['storm_category'].value_counts().sort_index()

print("\nEVENT DISTRIBUTION BY STORM CATEGORY")
print("="*60)
for category, count in event_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{category:25s}: {count:3d} events ({percentage:5.1f}%)")
print("="*60)

In [ ]:
# Detect significant storm periods (Kp >= 5)
storm_threshold = 5.0
storm_events = df[df['median'] >= storm_threshold].copy()

print(f"\nDETECTED STORM EVENTS (Kp >= {storm_threshold})")
print("="*60)
print(f"Total storm periods: {len(storm_events)}")
print(f"Percentage of time: {(len(storm_events)/len(df))*100:.1f}%")
print("\nStorm Events:")
print(storm_events[['Time (UTC)', 'median', 'maximum', 'storm_category']].to_string(index=False))

## 4. Storm Catalog Generation

Generate a detailed catalog of geomagnetic storms with event characteristics.

In [ ]:
# Create storm catalog with event grouping
storm_catalog = []

if len(storm_events) > 0:
    # Group consecutive storm periods (within 6 hours)
    storm_events = storm_events.sort_values('Time (UTC)')
    event_id = 1
    current_event = {
        'event_id': event_id,
        'start_time': storm_events.iloc[0]['Time (UTC)'],
        'end_time': storm_events.iloc[0]['Time (UTC)'],
        'peak_kp': storm_events.iloc[0]['median'],
        'max_kp': storm_events.iloc[0]['maximum'],
        'category': storm_events.iloc[0]['storm_category']
    }
    
    for i in range(1, len(storm_events)):
        time_diff = (storm_events.iloc[i]['Time (UTC)'] - current_event['end_time']).total_seconds() / 3600
        
        if time_diff <= 6:  # Same event if within 6 hours
            current_event['end_time'] = storm_events.iloc[i]['Time (UTC)']
            if storm_events.iloc[i]['median'] > current_event['peak_kp']:
                current_event['peak_kp'] = storm_events.iloc[i]['median']
                current_event['category'] = storm_events.iloc[i]['storm_category']
            if storm_events.iloc[i]['maximum'] > current_event['max_kp']:
                current_event['max_kp'] = storm_events.iloc[i]['maximum']
        else:  # New event
            # Calculate duration for completed event
            duration = (current_event['end_time'] - current_event['start_time']).total_seconds() / 3600
            current_event['duration_hours'] = duration
            storm_catalog.append(current_event)
            
            # Start new event
            event_id += 1
            current_event = {
                'event_id': event_id,
                'start_time': storm_events.iloc[i]['Time (UTC)'],
                'end_time': storm_events.iloc[i]['Time (UTC)'],
                'peak_kp': storm_events.iloc[i]['median'],
                'max_kp': storm_events.iloc[i]['maximum'],
                'category': storm_events.iloc[i]['storm_category']
            }
    
    # Add last event
    duration = (current_event['end_time'] - current_event['start_time']).total_seconds() / 3600
    current_event['duration_hours'] = duration
    storm_catalog.append(current_event)

# Convert to DataFrame
catalog_df = pd.DataFrame(storm_catalog)

print("\nSTORM CATALOG")
print("="*100)
if len(catalog_df) > 0:
    print(catalog_df.to_string(index=False))
    print("="*100)
    print(f"\nTotal distinct storm events: {len(catalog_df)}")
    print(f"Average storm duration: {catalog_df['duration_hours'].mean():.1f} hours")
    print(f"Peak Kp observed: {catalog_df['peak_kp'].max():.2f}")
else:
    print("No storm events detected in this period.")

## 5. Visualizations

### 5.1 Kp Time Series with Storm Events

In [ ]:
# Create comprehensive time series plot
fig, ax = plt.subplots(figsize=(16, 8))

# Plot median Kp
ax.plot(df['Time (UTC)'], df['median'], 
        marker='o', linestyle='-', linewidth=2.5, markersize=6,
        color='#2E86AB', label='Median Kp', zorder=3)

# Add min-max range as shaded area
ax.fill_between(df['Time (UTC)'], df['minimum'], df['maximum'], 
                alpha=0.2, color='#A23B72', label='Min-Max Range', zorder=1)

# Add storm threshold line
ax.axhline(y=5, color='orange', linestyle='--', linewidth=2, 
           label='Minor Storm Threshold (Kp=5)', zorder=2)
ax.axhline(y=7, color='red', linestyle='--', linewidth=2, 
           label='Strong Storm Threshold (Kp=7)', zorder=2)

# Highlight storm periods
if len(storm_events) > 0:
    ax.scatter(storm_events['Time (UTC)'], storm_events['median'],
               s=150, color='red', marker='*', 
               label='Storm Events', zorder=4, edgecolors='darkred', linewidths=1.5)

# Styling
ax.set_title('Geomagnetic Activity (Kp Index) Time Series\nNovember 12-15, 2025', 
             fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Time (UTC)', fontsize=14, fontweight='bold')
ax.set_ylabel('Kp Index', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11, framealpha=0.95)
ax.grid(True, alpha=0.3, linestyle=':')
plt.xticks(rotation=45, ha='right')
ax.set_ylim(0, max(df['maximum'].max() + 0.5, 8))

plt.tight_layout()
plt.savefig('../outputs/kp_timeseries_with_events.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Time series plot saved to outputs/kp_timeseries_with_events.png")

### 5.2 Kp Index Distribution

In [ ]:
# Create distribution plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogram with KDE
ax1.hist(df['median'], bins=20, alpha=0.7, color='#5E9EA0', edgecolor='black', linewidth=1.2)
ax1.axvline(df['median'].mean(), color='red', linestyle='--', linewidth=2.5, label=f'Mean: {df["median"].mean():.2f}')
ax1.axvline(df['median'].median(), color='green', linestyle='--', linewidth=2.5, label=f'Median: {df["median"].median():.2f}')
ax1.axvline(5, color='orange', linestyle=':', linewidth=2, label='Storm Threshold (Kp=5)')
ax1.set_title('Distribution of Kp Index (Median)', fontsize=16, fontweight='bold')
ax1.set_xlabel('Kp Index', fontsize=13)
ax1.set_ylabel('Frequency', fontsize=13)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, axis='y')

# Box plot
box_data = [df['median'], df['0.25-quantile'], df['0.75-quantile'], df['minimum'], df['maximum']]
positions = [1, 2, 3, 4, 5]
labels = ['Median', '25th %ile', '75th %ile', 'Minimum', 'Maximum']
bp = ax2.boxplot(box_data, positions=positions, labels=labels, patch_artist=True,
                  widths=0.6, showmeans=True, meanline=True)

# Color the boxes
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.axhline(5, color='orange', linestyle=':', linewidth=2, label='Storm Threshold')
ax2.set_title('Kp Index Quantile Distributions', fontsize=16, fontweight='bold')
ax2.set_ylabel('Kp Index', fontsize=13)
ax2.set_xlabel('Quantile Type', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.savefig('../outputs/kp_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Distribution plot saved to outputs/kp_distribution.png")

### 5.3 Storm Activity Distribution by Category

In [ ]:
# Create activity distribution plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
category_order = ['Quiet to Active', 'Minor Storm (G1)', 'Moderate Storm (G2)', 
                  'Strong Storm (G3)', 'Severe Storm (G4)', 'Extreme Storm (G5)']
event_counts_ordered = event_counts.reindex(category_order, fill_value=0)

colors_map = {
    'Quiet to Active': '#A8E6CF',
    'Minor Storm (G1)': '#FFD700',
    'Moderate Storm (G2)': '#FFA500',
    'Strong Storm (G3)': '#FF6347',
    'Severe Storm (G4)': '#DC143C',
    'Extreme Storm (G5)': '#8B0000'
}
bar_colors = [colors_map.get(cat, '#CCCCCC') for cat in event_counts_ordered.index]

bars = ax1.bar(range(len(event_counts_ordered)), event_counts_ordered.values, 
               color=bar_colors, edgecolor='black', linewidth=1.5)
ax1.set_xticks(range(len(event_counts_ordered)))
ax1.set_xticklabels(event_counts_ordered.index, rotation=45, ha='right')
ax1.set_title('Storm Activity Distribution by Category', fontsize=16, fontweight='bold')
ax1.set_ylabel('Number of Observations', fontsize=13)
ax1.set_xlabel('Storm Category', fontsize=13)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    if height > 0:
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontweight='bold', fontsize=11)

# Pie chart
pie_data = event_counts_ordered[event_counts_ordered > 0]
pie_colors = [colors_map.get(cat, '#CCCCCC') for cat in pie_data.index]

wedges, texts, autotexts = ax2.pie(pie_data.values, labels=pie_data.index, autopct='%1.1f%%',
                                     colors=pie_colors, startangle=90, 
                                     textprops={'fontsize': 10, 'fontweight': 'bold'})
ax2.set_title('Proportion of Storm Categories', fontsize=16, fontweight='bold')

# Make percentage text more visible
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(11)

plt.tight_layout()
plt.savefig('../outputs/storm_activity_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Activity distribution plot saved to outputs/storm_activity_distribution.png")

### 5.4 Storm Timeline

In [ ]:
# Create storm timeline visualization
if len(catalog_df) > 0:
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # Define colors for categories
    category_colors = {
        'Minor Storm (G1)': '#FFD700',
        'Moderate Storm (G2)': '#FFA500',
        'Strong Storm (G3)': '#FF6347',
        'Severe Storm (G4)': '#DC143C',
        'Extreme Storm (G5)': '#8B0000'
    }
    
    # Plot storm duration bars
    for idx, event in catalog_df.iterrows():
        color = category_colors.get(event['category'], '#808080')
        ax.barh(idx, event['duration_hours'], 
                left=event['start_time'].timestamp() / 3600,
                height=0.6, color=color, edgecolor='black', linewidth=1.5,
                alpha=0.8, label=event['category'] if event['category'] not in ax.get_legend_handles_labels()[1] else '')
        
        # Add peak Kp annotation
        mid_time = event['start_time'].timestamp() / 3600 + event['duration_hours'] / 2
        ax.text(mid_time, idx, f"Kp={event['peak_kp']:.1f}",
                ha='center', va='center', fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    # Format x-axis to show dates
    time_labels = pd.date_range(start=df['Time (UTC)'].min(), 
                                end=df['Time (UTC)'].max(), 
                                freq='12H')
    ax.set_xticks([t.timestamp() / 3600 for t in time_labels])
    ax.set_xticklabels([t.strftime('%m-%d %H:%M') for t in time_labels], rotation=45, ha='right')
    
    ax.set_yticks(range(len(catalog_df)))
    ax.set_yticklabels([f"Event {e['event_id']}" for _, e in catalog_df.iterrows()])
    
    ax.set_title('Geomagnetic Storm Timeline\nNovember 12-15, 2025', 
                 fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Time (UTC)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Storm Events', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=11, framealpha=0.95)
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig('../outputs/storm_timeline.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Storm timeline plot saved to outputs/storm_timeline.png")
else:
    print("No storm events to visualize in timeline.")

### 5.5 Probability Analysis

In [ ]:
# Analyze probability forecasts
prob_cols = ['prob 4-5', 'prob 5-6', 'prob 6-7', 'prob 7-8', 'prob >= 8']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Time series of probabilities
for col in prob_cols:
    ax1.plot(df['Time (UTC)'], df[col], marker='o', linestyle='-', 
             linewidth=2, markersize=5, label=col.replace('prob ', 'Kp '))

ax1.set_title('Forecast Probabilities for Different Kp Ranges Over Time', 
              fontsize=16, fontweight='bold')
ax1.set_xlabel('Time (UTC)', fontsize=13)
ax1.set_ylabel('Probability', fontsize=13)
ax1.legend(loc='upper right', fontsize=11)
ax1.grid(True, alpha=0.3)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
ax1.set_ylim(0, 1.05)

# Average probabilities
avg_probs = df[prob_cols].mean()
colors = ['#A8E6CF', '#FFD700', '#FFA500', '#FF6347', '#DC143C']
bars = ax2.bar(range(len(avg_probs)), avg_probs.values, color=colors, 
               edgecolor='black', linewidth=1.5, alpha=0.8)
ax2.set_xticks(range(len(avg_probs)))
ax2.set_xticklabels([col.replace('prob ', 'Kp ') for col in prob_cols])
ax2.set_title('Average Forecast Probability by Kp Range', fontsize=16, fontweight='bold')
ax2.set_ylabel('Average Probability', fontsize=13)
ax2.set_xlabel('Kp Range', fontsize=13)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}',
            ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig('../outputs/probability_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Probability analysis plot saved to outputs/probability_analysis.png")

## 6. Summary Statistics and Key Findings

In [ ]:
# Generate comprehensive summary
print("="*80)
print(" "*20 + "PHASE 3 ANALYSIS SUMMARY")
print("="*80)
print("\n📅 DATA PERIOD")
print("-" * 80)
print(f"  Start: {df['Time (UTC)'].min()}")
print(f"  End:   {df['Time (UTC)'].max()}")
print(f"  Duration: {(df['Time (UTC)'].max() - df['Time (UTC)'].min()).days + 1} days")
print(f"  Total observations: {len(df)}")

print("\n📊 KP INDEX STATISTICS")
print("-" * 80)
print(f"  Mean Kp:           {kp_median.mean():.3f}")
print(f"  Median Kp:         {kp_median.median():.3f}")
print(f"  Standard Dev:      {kp_median.std():.3f}")
print(f"  Range:             {kp_median.min():.3f} - {kp_median.max():.3f}")
print(f"  95th Percentile:   {kp_median.quantile(0.95):.3f}")

print("\n⚡ STORM ACTIVITY")
print("-" * 80)
print(f"  Storm periods (Kp ≥ 5):     {len(storm_events)}")
print(f"  Percentage of time in storm: {(len(storm_events)/len(df))*100:.1f}%")
if len(catalog_df) > 0:
    print(f"  Distinct storm events:       {len(catalog_df)}")
    print(f"  Peak Kp recorded:            {catalog_df['peak_kp'].max():.2f}")
    print(f"  Maximum Kp recorded:         {catalog_df['max_kp'].max():.2f}")
    print(f"  Average storm duration:      {catalog_df['duration_hours'].mean():.1f} hours")
    print(f"  Longest storm duration:      {catalog_df['duration_hours'].max():.1f} hours")

print("\n🎯 STORM CATEGORIES")
print("-" * 80)
for category, count in event_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {category:25s}: {count:3d} periods ({percentage:5.1f}%)")

print("\n🔍 KEY FINDINGS")
print("-" * 80)
print(f"  1. Peak Activity: The highest Kp index of {kp_median.max():.1f} was observed")
print(f"     on {df.loc[df['median'].idxmax(), 'Time (UTC)']}")
print(f"\n  2. Storm Frequency: {(len(storm_events)/len(df))*100:.1f}% of the observation period")
print(f"     experienced storm conditions (Kp ≥ 5)")
print(f"\n  3. Activity Level: Mean Kp of {kp_median.mean():.2f} indicates")
if kp_median.mean() < 3:
    print("     predominantly quiet geomagnetic conditions")
elif kp_median.mean() < 5:
    print("     moderate geomagnetic activity")
else:
    print("     elevated geomagnetic activity")

if len(catalog_df) > 0 and catalog_df['peak_kp'].max() >= 7:
    print(f"\n  4. Major Event: A Strong Storm (G3) with Kp = {catalog_df['peak_kp'].max():.1f}")
    print("     was detected, indicating significant geomagnetic disturbance")

print("\n" + "="*80)
print("✓ Phase 3 Analysis Complete!")
print("="*80)

## 7. Export Results

In [ ]:
# Export storm catalog to CSV
if len(catalog_df) > 0:
    catalog_df.to_csv('../outputs/storm_catalog.csv', index=False)
    print("✓ Storm catalog exported to outputs/storm_catalog.csv")

# Export summary statistics
summary_stats = pd.DataFrame({
    'Metric': list(stats.keys()),
    'Value': list(stats.values())
})
summary_stats.to_csv('../outputs/kp_statistics_summary.csv', index=False)
print("✓ Summary statistics exported to outputs/kp_statistics_summary.csv")

# Export event counts
event_summary = pd.DataFrame({
    'Category': event_counts.index,
    'Count': event_counts.values,
    'Percentage': [(count / len(df)) * 100 for count in event_counts.values]
})
event_summary.to_csv('../outputs/storm_category_distribution.csv', index=False)
print("✓ Storm category distribution exported to outputs/storm_category_distribution.csv")

print("\n" + "="*80)
print("All results exported successfully!")
print("="*80)

---

## Conclusions

This Phase 3 analysis successfully:

1. ✅ **Loaded and analyzed Kp index data** from November 12-15, 2025
2. ✅ **Calculated comprehensive statistics** for geomagnetic activity
3. ✅ **Detected and classified storm events** using standard thresholds
4. ✅ **Generated a storm catalog** with event characteristics
5. ✅ **Created visualizations** showing:
   - Kp time series with storm event markers
   - Distribution of Kp values across quantiles
   - Storm activity by category
   - Timeline of storm events
   - Probability forecast analysis
6. ✅ **Produced summary statistics** and key findings

### Next Steps (Phase 4+):
- Integrate additional data sources (solar wind, X-ray flux)
- Perform correlation analysis between different parameters
- Analyze technology impact during storm periods
- Build predictive models for storm forecasting

---

**Analysis Date:** November 2025  
**Notebook:** 03-correlation-analysis.ipynb  
**Status:** ✅ Complete